# Finite Capacity Region with Blocking vs M/M/1

This example shows that an FCR with blocking (waitq) around a single queue behaves identically to a standard M/M/1 queue, since jobs simply wait when the region is "full" (no actual capacity limit effect).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
// Parameters
val arrivalRate = 0.5
val serviceRate = 1.0

In [ ]:
// Model 1: Queue with FCR (blocking)
val model1 = Network("FCR Blocking")

val source1 = Source(model1, "Source")
val queue1 = Queue(model1, "Queue", SchedStrategy.FCFS)
val sink1 = Sink(model1, "Sink")

val jobclass1 = OpenClass(model1, "Class1", 0)

source1.setArrival(jobclass1, Exp(arrivalRate))
queue1.setService(jobclass1, Exp(serviceRate))

val P1 = model1.initRoutingMatrix()
P1.set(jobclass1, jobclass1, source1, queue1, 1.0)
P1.set(jobclass1, jobclass1, queue1, sink1, 1.0)
model1.link(P1)

// Add FCR with blocking - jobs wait when region is full
val fcr = model1.addRegion(listOf(queue1))
fcr.setGlobalMaxJobs(10)
fcr.setDropRule(jobclass1, false)  // false = block (wait)

In [ ]:
// Model 2: Standard M/M/1 (no FCR)
val model2 = Network("M/M/1")

val source2 = Source(model2, "Source")
val queue2 = Queue(model2, "Queue", SchedStrategy.FCFS)
val sink2 = Sink(model2, "Sink")

val jobclass2 = OpenClass(model2, "Class1", 0)

source2.setArrival(jobclass2, Exp(arrivalRate))
queue2.setService(jobclass2, Exp(serviceRate))

val P2 = model2.initRoutingMatrix()
P2.set(jobclass2, jobclass2, source2, queue2, 1.0)
P2.set(jobclass2, jobclass2, queue2, sink2, 1.0)
model2.link(P2)

In [ ]:
// Solve both models and compare results
val solver1 = JMT(model1, "seed", 23000, "samples", 100000)
val avgTable1 = solver1.avgNodeTable
println("Model 1: FCR with Blocking")
avgTable1.print()

In [ ]:
val solver2 = JMT(model2, "seed", 23000, "samples", 100000)
val avgTable2 = solver2.avgNodeTable
println("\nModel 2: Standard M/M/1")
avgTable2.print()

## Explanation

This example demonstrates that an FCR with blocking behavior is equivalent to a standard queue:

1. **FCR with Blocking**: A queue inside a Finite Capacity Region with blocking policy (`setDropRule(false)`).

2. **Standard M/M/1**: A queue without any capacity constraints.

Both models produce identical results because:
- When the FCR is full, arriving jobs wait (block) in the queue
- The M/M/1 queue naturally allows jobs to wait
- The capacity limit has no practical effect since jobs are never rejected

This validates that FCRs with blocking (`setDropRule(false)`) allow jobs to wait rather than being lost.